# Notebook 6: Univariate Time Series

In this notebook we develop three univariate time series models using Arima, Auto Arima and Prophet to forecast predictions for Food Insecurity rates in a single state until 2026. We then develop a method to generate these predictions for each state in the country, save the predicitons to a dataframe and then concatenate all dataframes together into a single dataframe. Ultimately, given the poor accuracy of the Arima and Auto-Arima models, we do not keep these models and instead use the predictions from the Prophet model as our forecasted Food Insecurity rates. We use this exported dataframe with our forecasts in Tableau, which is then hosted on our Streamlit app, the script for which can be found in the 'food_ins.py' file in this repo.

### Imports and Cleaning the data for use in Time Series

In [3]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error
import datetime
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from fbprophet import 
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('./data/time_series/df_ts_state_mean_years.csv')

In [6]:
#Dropping unnecessary columns, transposing and resetting index
df = df.drop(columns = 'fips', axis = 1)

df = df.rename(columns = {"state_name": ""})

df_t = df.T

df_t.reset_index()

In [7]:
#Exporting to CSV
df_t.to_csv('./data/time_series/df_ts_rev.csv')

In [8]:
#Reimporting a renamed file
df_rev = pd.read_csv('./data/time_series/df_ts_rev_fin_yr10.csv')

In [9]:
df_rev.head()

,Unnamed: 0,time,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,0,2010-01-01,20.379104,15.021739,20.746667,19.257333,17.989655,14.126563,11.712500,12.366667,15.800000,17.605970,19.801258,14.025000,16.184091,14.561765,16.042391,12.960606,13.351429,18.341667,16.045312,16.106250,11.739130,11.071429,18.119277,11.778161,22.310976,17.210435,13.689286,11.520430,14.852941,10.450000,12.766667,17.593939,12.848387,19.279000,8.235849,17.586364,17.349351,17.436111,13.849254,13.880000,21.360870,12.238462,18.925263,17.921654,15.944828,13.778571,14.051880,16.066667,15.025455,13.350000,11.917391
1,1,2011-01-01,19.292537,14.573913,19.553333,18.981333,17.800000,14.257813,12.050000,12.000000,16.459510,17.788060,19.516981,13.675000,16.522727,13.556863,14.607609,12.563636,12.995238,17.409167,16.182812,15.350000,11.226087,10.914286,16.100000,10.491954,22.564634,16.237391,14.194643,11.400000,15.794118,9.960000,12.423810,17.563636,12.327419,18.585000,7.683019,16.414773,16.536364,16.391667,13.225373,13.920000,20.404348,12.096923,16.754737,17.421260,16.275862,12.864286,13.425564,15.679487,15.512727,12.484722,12.134783
2,2,2012-01-01,19.497412,14.694528,19.090460,19.714232,17.426683,15.542930,14.532038,13.230666,15.715312,18.674549,20.023413,14.554738,17.301741,15.210277,16.326573,12.919010,15.089658,17.153329,17.147063,15.707114,13.403591,12.294734,17.919770,11.373916,21.435536,17.120004,14.928887,13.679279,17.356009,10.613764,13.951574,20.064124,14.657905,19.296320,7.369154,17.793839,17.244326,17.925498,14.867511,15.319100,18.597291,12.520599,17.603121,18.745160,16.748769,13.050120,12.680996,16.127143,15.269241,13.032920,11.989623
3,3,2013-01-01,18.569592,13.969532,17.847161,19.430470,16.244368,14.568059,13.883338,13.030318,14.476778,17.850177,18.897181,14.169228,15.795925,14.242902,15.658524,12.677502,14.790854,16.676046,16.910432,15.498137,13.139463,11.883582,16.796413,10.746515,22.305076,17.121090,14.583029,13.370879,16.821370,10.925118,12.994284,18.561158,14.096593,18.599840,7.738398,17.170230,17.203957,16.746368,14.327769,14.656876,17.964432,12.342894,17.065653,18.319248,15.544147,13.359190,12.137138,14.955172,14.998278,12.649662,13.004587
4,4,2014-01-01,18.800626,14.249142,17.526119,19.743033,14.952682,13.914748,13.580065,13.212005,14.995769,16.956854,18.732573,13.812859,15.558861,13.624915,15.415996,12.595392,14.489849,16.434164,16.812346,15.515297,12.775405,11.468253,16.405336,10.566787,22.724940,17.041708,14.228229,13.216906,15.812491,10.820131,12.380133,17.307929,13.942814,18.297411,7.800739,16.869003,17.001128,15.751393,14.159137,14.440172,17.059423,12.442048,17.053937,17.593981,14.596301,13.566219,11.895906,14.601072,15.774113,12.433148,12.781083


In [10]:
#Final drop
df_rev = df_rev.drop(columns = 'Unnamed: 0', axis = 1)

## Modeling Preprocessing - Make Train and Validation DFs

In [12]:
## https://stackoverflow.com/questions/26921651/how-to-delete-the-last-row-of-data-of-a-pandas-dataframe

df_train = pd.DataFrame(df_rev.drop(df_rev.tail(2).index))

In [13]:
df_val = pd.DataFrame(df_rev.drop(df.head(7).index))

In [14]:
df_val.head()

,time,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
7,2017-01-01,18.092537,14.865217,16.160000,18.138667,13.065517,11.062500,10.7250,11.466667,11.9,15.507463,16.447799,11.425,13.211364,11.579412,12.470652,11.023232,12.109524,15.709167,17.009375,14.11875,10.343478,9.257143,12.860241,9.281609,21.203659,14.718261,12.653571,11.405376,12.135294,9.39,10.014286,15.900000,11.029032,15.947,6.986792,13.692045,15.370130,13.577778,11.983582,11.16,16.086957,11.755385,14.093684,14.861417,13.958621,11.685714,12.034586,13.061538,14.000000,10.233333,12.434783
8,2018-01-01,17.155224,14.778261,15.533333,17.758667,12.389655,10.560937,10.4750,11.466667,11.4,14.641791,15.710692,10.575,12.543182,11.038235,11.870652,10.614141,11.836190,15.429167,16.557813,13.38750,10.139130,9.021429,12.887952,9.163218,20.345122,14.064348,12.050000,11.278495,11.341176,9.11,9.638095,15.963636,10.904839,15.419,6.473585,13.456818,15.031169,12.955556,11.483582,10.42,15.378261,12.004615,13.327368,14.470079,13.413793,11.500000,11.742857,12.423077,13.803636,9.612500,11.786957
9,2019-01-01,17.719403,14.265217,15.753333,18.238667,11.910345,11.039063,10.7625,11.366667,10.7,13.944776,14.847799,10.850,11.577273,10.918627,12.586957,9.000000,12.293333,16.362500,16.535938,13.51875,11.091304,8.478571,13.660241,8.600000,19.312195,14.190435,10.860714,12.017204,12.370588,9.38,8.757143,15.487879,10.888710,15.147,6.466038,13.621591,15.001299,13.133333,10.852239,9.22,12.836957,11.343077,14.424211,15.304331,12.555172,10.857143,11.111278,12.853846,14.085455,8.950000,12.256522


## Model 1: ARIMA

In [16]:
## Help and base code comes from: https://www.youtube.com/watch?v=axjgEgBgIY0

In [17]:
au = df_train[['time', 'Georgia']]
au_v = df_val[['time', 'Georgia']]

au.set_index('time', inplace=True)
au_v.set_index('time', inplace=True)

index_5_years = pd.date_range(au.index[-1], freq = 'AS', periods = 5)

In [18]:
mod_arima = ARIMA(au, order=(1,1,1))
model_arima_fit = mod_arima.fit()

fcast1 = model_arima_fit.forecast(5)[0]

fcast1 = pd.Series(fcast1, index=index_5_years)
fcast1 = fcast1.rename('Arima');

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [19]:
# fig, ax = plt.subplots(figsize =(15,5))
# chart = sns.lineplot(x='time', y = 'Georgia', data = au)
# chart.set_title('AU')
# fcast1.plot(ax=ax, color = 'green', marker = 'o', legend = True)
# au.plot(ax=ax, color = 'blue', marker = 'o', legend = True)
# au_v.plot(ax=ax, color = 'orange', marker = 'o', legend = True)

## Model 2: Auto-Arima

In [20]:
## Help and base code comes from: https://www.youtube.com/watch?v=axjgEgBgIY0

In [22]:
auto_arima_mod = pm.auto_arima(au, seasonal = False, m = 4)

fcast2 = auto_arima_mod.predict(5)

fcast2 = pd.Series(fcast2, index=index_5_years)
fcast2 = fcast2.rename('Auto_Arima')

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_validation.py:62: UserWarning: m (4) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(1,0,2)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/ae-j/opt/anaconda3/lib/python3.8/site-packa

In [23]:
# fig, ax = plt.subplots(figsize =(15,5))
# chart = sns.lineplot(x='time', y = 'Georgia', data = au)
# chart.set_title('AU')
# fcast2.plot(ax=ax, color = 'green', marker = 'o', legend = True)
# au.plot(ax=ax, color = 'blue', marker = 'o', legend = True)
# au_v.plot(ax=ax, color = 'orange', marker = 'o', legend = True)

## Model 3: Prophet

In [ ]:
## Help and base code comes from: https://www.youtube.com/watch?v=axjgEgBgIY0

In [24]:
au.head()

,Georgia
time,
2010-01-01,19.801258
2011-01-01,19.516981
2012-01-01,20.023413
2013-01-01,18.897181
2014-01-01,18.732573


In [26]:
## Copying and renaming 
au_c = df_train[['time', 'Georgia']].copy()
au_c.columns = ['ds', 'y']
au_c['ds'] = pd.to_datetime(au_c['ds'])

model_p = Prophet()
model_p.fit(au_c)

future = model_p.make_future_dataframe(5, freq = 'Y')

#make predictions

fcast3 = model_p.predict(future)

fcast3 = pd.Series(fcast3['yhat'].values, name = 'Prophet', index = fcast3['ds'])

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 5.


In [27]:
# fig, ax = plt.subplots(figsize =(15,5))
# chart = sns.lineplot(x='time', y = 'Georgia', data = au)
# chart.set_title('AU')
# fcast3.plot(ax=ax, color = 'green', marker = 'o', legend = True)
# au.plot(ax=ax, color = 'blue', marker = 'o', legend = True)
# au_v.plot(ax=ax, color = 'orange', marker = 'o', legend = True)

In [28]:
# print(f'The MSE of Prophet is: {mean_squared_error(au_v['Georgia'].values, fcast3.values, squared= False)}.')

## Model Aggregation 1: Running Arima and Auto-Arima Models Side-By-Side

In [20]:
## Help and base code comes from: https://www.youtube.com/watch?v=axjgEgBgIY0

In [29]:
df_train.head()

,time,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2010-01-01,20.379104,15.021739,20.746667,19.257333,17.989655,14.126563,11.712500,12.366667,15.800000,17.605970,19.801258,14.025000,16.184091,14.561765,16.042391,12.960606,13.351429,18.341667,16.045312,16.106250,11.739130,11.071429,18.119277,11.778161,22.310976,17.210435,13.689286,11.520430,14.852941,10.450000,12.766667,17.593939,12.848387,19.279000,8.235849,17.586364,17.349351,17.436111,13.849254,13.880000,21.360870,12.238462,18.925263,17.921654,15.944828,13.778571,14.051880,16.066667,15.025455,13.350000,11.917391
1,2011-01-01,19.292537,14.573913,19.553333,18.981333,17.800000,14.257813,12.050000,12.000000,16.459510,17.788060,19.516981,13.675000,16.522727,13.556863,14.607609,12.563636,12.995238,17.409167,16.182812,15.350000,11.226087,10.914286,16.100000,10.491954,22.564634,16.237391,14.194643,11.400000,15.794118,9.960000,12.423810,17.563636,12.327419,18.585000,7.683019,16.414773,16.536364,16.391667,13.225373,13.920000,20.404348,12.096923,16.754737,17.421260,16.275862,12.864286,13.425564,15.679487,15.512727,12.484722,12.134783
2,2012-01-01,19.497412,14.694528,19.090460,19.714232,17.426683,15.542930,14.532038,13.230666,15.715312,18.674549,20.023413,14.554738,17.301741,15.210277,16.326573,12.919010,15.089658,17.153329,17.147063,15.707114,13.403591,12.294734,17.919770,11.373916,21.435536,17.120004,14.928887,13.679279,17.356009,10.613764,13.951574,20.064124,14.657905,19.296320,7.369154,17.793839,17.244326,17.925498,14.867511,15.319100,18.597291,12.520599,17.603121,18.745160,16.748769,13.050120,12.680996,16.127143,15.269241,13.032920,11.989623
3,2013-01-01,18.569592,13.969532,17.847161,19.430470,16.244368,14.568059,13.883338,13.030318,14.476778,17.850177,18.897181,14.169228,15.795925,14.242902,15.658524,12.677502,14.790854,16.676046,16.910432,15.498137,13.139463,11.883582,16.796413,10.746515,22.305076,17.121090,14.583029,13.370879,16.821370,10.925118,12.994284,18.561158,14.096593,18.599840,7.738398,17.170230,17.203957,16.746368,14.327769,14.656876,17.964432,12.342894,17.065653,18.319248,15.544147,13.359190,12.137138,14.955172,14.998278,12.649662,13.004587
4,2014-01-01,18.800626,14.249142,17.526119,19.743033,14.952682,13.914748,13.580065,13.212005,14.995769,16.956854,18.732573,13.812859,15.558861,13.624915,15.415996,12.595392,14.489849,16.434164,16.812346,15.515297,12.775405,11.468253,16.405336,10.566787,22.724940,17.041708,14.228229,13.216906,15.812491,10.820131,12.380133,17.307929,13.942814,18.297411,7.800739,16.869003,17.001128,15.751393,14.159137,14.440172,17.059423,12.442048,17.053937,17.593981,14.596301,13.566219,11.895906,14.601072,15.774113,12.433148,12.781083


In [30]:
## Creating a states list for proof that it can work on multiple states.
states = ['Georgia', 'Alabama']

for s in states:
    #training data
    train_data = df_train[['time', s]]
    
    #valid data
    valid_data = df_val[['time', s]]
    
    #all data
    all_data = df_rev[['time', s]]
    
    #Set time column to index
    train_data.set_index('time', inplace=True)
    valid_data.set_index('time', inplace=True)
    valid_data.columns = ['Valid Data'] ##To see it on the graph
    all_data.set_index('time', inplace=True)
    
    #Set valid index for 5 years
    index_7_years = pd.date_range(train_data.index[-1], freq = 'AS', periods = 7)
    
    #Future index - 5 years
    future_7_years = pd.date_range(valid_data.index[-1], freq = 'AS', periods = 7)
    
    ## Tricky bit of code in order to basically reset the forecasts from the previous state. 
    # Otherwise, if a state fails to work in the Arima model, the forecast from the previous state is going to be passed in.
    
    #Drop all tables:
    try:
        del t_fcast1
        del t_fcast2
        #del t_fcast3
    
        del f_fcast1
        del f_fcast2
        #del f_fcast3
    except:
        print("")
    
    try:
        del t_fcast1
        del t_fcast2
        del t_fcast3
    except:
        print("")

## Arima Model ##

#Arima Validation Phase 

    try:
        model_arima = ARIMA(train_data, order=(1,1,1))
        model_arima_fit = model_arima.fit()

        t_fcast1 = model_arima_fit.forecast(7)[0]

        t_fcast1 = pd.Series(fcast1, index=index_7_years)
        t_fcast1 = t_fcast1.rename('Arima')
    except:
        print(s, 'Arima Train Error')
          

#Arima Future Phase
            
    try:
        model_arima = ARIMA(all_data, order=(1,1,1))
        model_arima_fit = model_arima.fit()

        f_fcast1 = model_arima_fit.forecast(7)[0]

        f_fcast1 = pd.Series(fcast1, index=future_7_years)
        f_fcast1 = f_fcast1.rename('Future_Arima')
    except:
        print(s, 'Arima Future Error')
        

##### Auto-Arima #####


#Auto Arima Valid phase
    try:
        auto_arima_model = pm.auto_arima(train_data, seasonal = False, m = 4)

        t_fcast2 = auto_arima_model.predict(7)
        t_fcast2 = pd.Series(t_fcast2, index=index_7_years)
        t_fcast2 = t_fcast2.rename('Auto_Arima')
    except:
        print(s, 'Auto Arima Train Error')
        

#Auto Arima Future phase
    try:
        auto_arima_model = pm.auto_arima(all_data, seasonal = False, m = 4)

        f_fcast2 = auto_arima_model.predict(7)
        f_fcast2 = pd.Series(f_fcast2, index=future_7_years)
        f_fcast2 = f_fcast2.rename('Future_Auto_Arima')
    except:
        print(s, 'Auto Arima Future Error')
    
#Plotting
# fig, ax = plt.subplots(figsize =(15,5))
# chart = sns.lineplot(x='time', y = s, data = train_data)
# chart.set_title(s)
# valid_data.plot(ax=ax, color = 'blue', marker = 'o', legend = True)
# #Plotting val
# try:
#     t_fcast1.plot(ax=ax, color = 'red', marker = 'o', legend = True)
# except:
#     print('')
# try:
#     t_fcast2.plot(ax=ax, color = 'green', marker = 'o', legend = True)
# except:
#     print('')
    
# #plotting future
# try:
#      f_fcast1.plot(ax=ax, color = 'red', marker = 'v', legend = True)
# except:
#     print('')
# try:
#     f_fcast2.plot(ax=ax, color = 'green', marker = 'v', legend = True)
# except:
#     print('')

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/ae-j/opt/anaconda3/lib/

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

## Model Aggregation 2: Arima, Auto Arima and Prophet 

In [32]:
df_train.head()

,time,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2010-01-01,20.379104,15.021739,20.746667,19.257333,17.989655,14.126563,11.712500,12.366667,15.800000,17.605970,19.801258,14.025000,16.184091,14.561765,16.042391,12.960606,13.351429,18.341667,16.045312,16.106250,11.739130,11.071429,18.119277,11.778161,22.310976,17.210435,13.689286,11.520430,14.852941,10.450000,12.766667,17.593939,12.848387,19.279000,8.235849,17.586364,17.349351,17.436111,13.849254,13.880000,21.360870,12.238462,18.925263,17.921654,15.944828,13.778571,14.051880,16.066667,15.025455,13.350000,11.917391
1,2011-01-01,19.292537,14.573913,19.553333,18.981333,17.800000,14.257813,12.050000,12.000000,16.459510,17.788060,19.516981,13.675000,16.522727,13.556863,14.607609,12.563636,12.995238,17.409167,16.182812,15.350000,11.226087,10.914286,16.100000,10.491954,22.564634,16.237391,14.194643,11.400000,15.794118,9.960000,12.423810,17.563636,12.327419,18.585000,7.683019,16.414773,16.536364,16.391667,13.225373,13.920000,20.404348,12.096923,16.754737,17.421260,16.275862,12.864286,13.425564,15.679487,15.512727,12.484722,12.134783
2,2012-01-01,19.497412,14.694528,19.090460,19.714232,17.426683,15.542930,14.532038,13.230666,15.715312,18.674549,20.023413,14.554738,17.301741,15.210277,16.326573,12.919010,15.089658,17.153329,17.147063,15.707114,13.403591,12.294734,17.919770,11.373916,21.435536,17.120004,14.928887,13.679279,17.356009,10.613764,13.951574,20.064124,14.657905,19.296320,7.369154,17.793839,17.244326,17.925498,14.867511,15.319100,18.597291,12.520599,17.603121,18.745160,16.748769,13.050120,12.680996,16.127143,15.269241,13.032920,11.989623
3,2013-01-01,18.569592,13.969532,17.847161,19.430470,16.244368,14.568059,13.883338,13.030318,14.476778,17.850177,18.897181,14.169228,15.795925,14.242902,15.658524,12.677502,14.790854,16.676046,16.910432,15.498137,13.139463,11.883582,16.796413,10.746515,22.305076,17.121090,14.583029,13.370879,16.821370,10.925118,12.994284,18.561158,14.096593,18.599840,7.738398,17.170230,17.203957,16.746368,14.327769,14.656876,17.964432,12.342894,17.065653,18.319248,15.544147,13.359190,12.137138,14.955172,14.998278,12.649662,13.004587
4,2014-01-01,18.800626,14.249142,17.526119,19.743033,14.952682,13.914748,13.580065,13.212005,14.995769,16.956854,18.732573,13.812859,15.558861,13.624915,15.415996,12.595392,14.489849,16.434164,16.812346,15.515297,12.775405,11.468253,16.405336,10.566787,22.724940,17.041708,14.228229,13.216906,15.812491,10.820131,12.380133,17.307929,13.942814,18.297411,7.800739,16.869003,17.001128,15.751393,14.159137,14.440172,17.059423,12.442048,17.053937,17.593981,14.596301,13.566219,11.895906,14.601072,15.774113,12.433148,12.781083


In [35]:
## Creating a full states list:

states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","District of Columbia", "Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

for s in states:
    #training data
    train_data = df_train[['time', s]]
    
    #valid data
    valid_data = df_val[['time', s]]
    
    #all data
    all_data = df_rev[['time', s]]
    
    #Set time column to index
    train_data.set_index('time', inplace=True)
    valid_data.set_index('time', inplace=True)
    valid_data.columns = ['Valid Data'] ##To see it on the graph
    all_data.set_index('time', inplace=True)
    
    #Set valid index for 7 years
    index_7_years = pd.date_range(train_data.index[-1], freq = 'AS', periods = 7)
    
    #Future index - 7 years
    future_7_years = pd.date_range(valid_data.index[-1], freq = 'AS', periods = 7)
    
    ## Tricky bit of code in order to basically reset the forecasts from the previous state. 
    # Otherwise, if a state fails to work in the Arima model, the forecast from the previous state is going to be passed in.
    
    #Drop all tables:
    try:
        del t_fcast1
        del t_fcast2
        del t_fcast3
#         del t_fcast4
    
        del f_fcast1
        del f_fcast2
        del f_fcast3
#         del f_fcast4
    except:
        print("")
    
    try:
        del t_fcast1
        del t_fcast2
        del t_fcast3
    except:
        print("")

##################################################################################################################
################################################ Arima #########################################################
##################################################################################################################


############################################# Arima Validation Phase ###################################################


    try:
        model_arima = ARIMA(train_data, order=(1,1,1))
        model_arima_fit = model_arima.fit()

        t_fcast1 = model_arima_fit.forecast(7)[0]

        t_fcast1 = pd.Series(fcast1, index=index_7_years)
        t_fcast1 = t_fcast1.rename('Arima')
    except:
        print(s, 'Arima Train Error')
          

################################################  Arima Future Phase ###################################################

            
    try:
        model_arima = ARIMA(all_data, order=(1,1,1))
        model_arima_fit = model_arima.fit()

        f_fcast1 = model_arima_fit.forecast(7)[0]

        f_fcast1 = pd.Series(fcast1, index=future_7_years)
        f_fcast1 = f_fcast1.rename('Future_Arima')
    except:
        print(s, 'Arima Future Error')
        

##################################################################################################################
################################################ Auto Arima ######################################################
##################################################################################################################


################################################  Auto Arima Validation Phase ###################################################

    try:
        auto_arima_model = pm.auto_arima(train_data, seasonal = False, m = 4)

        t_fcast2 = auto_arima_model.predict(7)
        t_fcast2 = pd.Series(t_fcast2, index=index_7_years)
        t_fcast2 = t_fcast2.rename('Auto_Arima')
    except:
        print(s, 'Auto Arima Train Error')
        

################################################  Auto Arima Future Phase ###################################################
   
    try:
        auto_arima_model = pm.auto_arima(all_data, seasonal = False, m = 4)

        f_fcast2 = auto_arima_model.predict(7)
        f_fcast2 = pd.Series(f_fcast2, index=future_7_years)
        f_fcast2 = f_fcast2.rename('Future_Auto_Arima')
    except:
        print(s, 'Auto Arima Future Error')
    
##################################################################################################################
################################################ Prophet #########################################################
##################################################################################################################

    #Expected column names Train
    train_data3 = df_train[['time', s]].copy()
    train_data3.columns = ['ds', 'y']
    train_data3['ds'] = pd.to_datetime(train_data3['ds'])

    #Expected column names Tall
    all_data3 = df_rev[['time', s]].copy()
    all_data3.columns = ['ds', 'y']
    all_data3['ds'] = pd.to_datetime(all_data3['ds'])
    
################################################  Prophet Validation Phase ###################################################
        
    model_p = Prophet()
    model_p.fit(train_data3)

    ##make validation index 
    val = model_p.make_future_dataframe(7, freq = 'Y')
    
    #make predictions
    t_fcast3 = model_p.predict(val)
    t_fcast3 = pd.Series(t_fcast3['yhat'].values, name = 'Prophet', index = t_fcast3['ds'])

################################################  Prophet Future Phase #######################################################
    
    model_pf = Prophet()
    model_pf.fit(all_data3)

    ##make validation index 
    future = model_pf.make_future_dataframe(7, freq = 'Y')
    
    #make predictions
    f_fcast3 = model_pf.predict(future)
    f_fcast3 = pd.Series(f_fcast3['yhat'].values, name = 'Future_Prophet', index = f_fcast3['ds'])


##################################################################################################################
################################################ Regression #######################################################
##################################################################################################################

#     train_data4 = df_train[['time', s]]
#     all_data4 = df_rev[['time', s]]
    
#     #ordinal data conversion
#     train_data4['MonthNumberOrdinal'] = train_data4['time']
#     train_data4['MonthNumberOrdinal'] = pd.to_datetime(train_data4['MonthNumberOrdinal'].map(pd.datetime.toordinal))

#     all_data4['MonthNumberOrdinal'] = all_data4['time']
#     all_data4['MonthNumberOrdinal'] = pd.to_datetime(all_data4['MonthNumberOrdinal'].map(pd.datetime.toordinal))

#     #Set valid index for 5 years
#     index_7_years['ForecastDate_Ordinal'] = pd.to_datetime(index_7_years['ds'].map(datetime.toordinal))
#     future_7_years['ForecastDate_Ordinal'] = pd.to_datetime(future_7_years['ds'].map(datetime.toordinal))
   
#     ############ Regression Validation Phase ################
#     ln = LinearRegression()
#     ln.fit(train_data4['MonthNumberOrdinal'].values.reshape(-1, 1), train_data4[s].values)
                                                            
#     #predictions
#     t_fcast4 = ln.predict(index_7_years['Forecast_Ordinal'].values.reshape(-1, 1))
    
#     #passing on same index
#     t_fcast4 = pd.Series(t_fcast4, index=index_7_years)
#     t_fcast4 = t_fcast4.rename('Regression')
                                                            
################################################  DF Prophet #######################################################

#     ln2 = LinearRegression()
#     ln2.fit(all_data['MonthNumberOrdinal'].values.reshape(-1, 1), all_data[s].values)
                                                            
#     #predictions
#     f_fcast4 = ln2.predict(future_7_years['Forecast_Ordinal'].values.reshape(-1, 1))                                          
    
#     #passing on same index
#     f_fcast4 = pd.Series(f_fcast4, index=future_7_years)
#     f_fcast4 = f_fcast4.rename('Future_Regression')

##################################################################################################################
################################################ RMSE ############################################################
##################################################################################################################
    
#     t_fcast1_mse = mean_squared_error(valid_data[s].values, t_fcast1[s], squared=False)
#     t_fcast2_mse = mean_squared_error(valid_data[s].values, t_fcast2[s], squared=False)                                                        
#     t_fcast3_mse = mean_squared_error(valid_data[s].values, t_fcast3[s], squared=False)
# #     t_fcast4_mse = mean_squared_error(valid_data[s].values, t_fcast4[s], squared=False)
                                                            
##################################################################################################################
################################################ Plotting ######################################################
##################################################################################################################

#     fig, ax = plt.subplots(figsize =(15,5))
#     chart = sns.lineplot(x='time', y = s, data = train_data)
#     chart.set_title(s)
#     valid_data.plot(ax=ax, color = 'blue', marker = 'o', legend = True)
#     #Plotting val
#     try:
#         t_fcast1.plot(ax=ax, color = 'red', marker = 'o', legend = True)
#     except:
#         print('')
#     try:
#         t_fcast2.plot(ax=ax, color = 'green', marker = 'o', legend = True)
#     except:
#         print('')

    #plotting future
#     try:
# # #          f_fcast1.plot(ax=ax, color = 'red', marker = 'v', legend = True)
#     except:
#         print('')
#     try:
# #         f_fcast2.plot(ax=ax, color = 'green', marker = 'v', legend = True)
#     except:
#         print('')
    
#     t_fcast3.plot(ax=ax, color = 'orange', marker = 'o', legend = True)
#     f_fcast3.plot(ax=ax, color = 'orange', marker = 'o', legend = True)
# #     t_fcast4.plot(ax=ax, color = 'black', marker = 'o', legend = True)
# #     f_fcast4.plot(ax=ax, color = 'black', marker = 'o', legend = True)

##################################################################################################################
################################################ Saving into DataFrames ##########################################
##################################################################################################################

################################################  DF Arima #######################################################

    try:
        #Creating df for forecast1
        t_fcast1 = t_fcast1.reset_index()
        t_fcast1.columns = ['Year', 'Arima ForecastValue Validation']
        
        f_fcast1 = f_fcast1.reset_index()
        f_fcast1.columns = ['Year', 'Arima ForecastValue Future']
        
        #Extra Columns
        t_fcast1['Arima ForecastValue Future'] = np.nan
        f_fcast1['Arima ForecastValue Validation'] = np.nan
        
        #Reordering
        t_fcast1 = t_fcast1[['Year', 'Arima ForecastValue Future', 'Arima ForecastValue Validation']]
        
        # Joining them togther
        df_fcast1 = pd.concat([t_fcast1, f_fcast1], axis = 0)
        df_fcast1['State'] = s
        df_fcast1['ML Method'] = 'Arima'
#         df_fcast1['Arima MSE'] = t_fcast1_mse
    
    except:
        df_fcast1 = pd.DataFrame({'Year': [np.nan], 'Arima ForecastValue Future': [np.nan], 'Arima ForecastValue Validation': [np.nan], 'State': [s], 'ML Method': ['Arima']})

################################################  DF Auto Arima #######################################################
     
    try:
        #Creating df for forecast2
        t_fcast2 = t_fcast2.reset_index()
        t_fcast2.columns = ['Year', 'Auto Arima ForecastValue Validation']
        
        f_fcast2 = f_fcast2.reset_index()
        f_fcast2.columns = ['Year', 'Auto Arima ForecastValue Future']
        
        #Extra Columns
        t_fcast2['Auto Arima ForecastValue Future'] = np.nan
        f_fcast2['Auto Arima ForecastValue Validation'] = np.nan
        
        #Reordering
        t_fcast2 = t_fcast2[['Year', 'Auto Arima ForecastValue Future', 'Auto Arima ForecastValue Validation']]
        
        # Joining them togther
        df_fcast2 = pd.concat([t_fcast2, f_fcast2], axis = 0)
        df_fcast2['State'] = s
        df_fcast2['ML Method'] = 'Auto Arima'
#         df_fcast2['Auto Arima MSE'] = t_fcast2_mse
    
    except:
        df_fcast2 = pd.DataFrame({'Year': [np.nan], 'Auto Arima ForecastValue Future': [np.nan], 'Auto Arima ForecastValue Validation': [np.nan], 'State': [s], 'ML Method': ['Auto Arima']})

################################################  DF Prophet #######################################################
     
    try:
        #Creating df for forecast3
        t_fcast3 = t_fcast3.reset_index()
        t_fcast3.columns = ['Year', 'Prophet ForecastValue Validation']
    
        f_fcast3 = f_fcast3.reset_index()
        f_fcast3.columns = ['Year', 'Prophet ForecastValue Future']
        
        #Extra Columns
        t_fcast3['Prophet ForecastValue Future'] = np.nan
        f_fcast3['Prophet ForecastValue Validation'] = np.nan
        
        #Reordering
        t_fcast3 = t_fcast3[['Year', 'Prophet ForecastValue Future', 'Prophet ForecastValue Validation']]
        
        # Joining them togther
        df_fcast3 = pd.concat([t_fcast3, f_fcast3], axis = 0)
        df_fcast3['State'] = s
        df_fcast3['ML Method'] = 'Prophet'
#         df_fcast3['Prophet MSE'] = t_fcast3_mse

    except:
        df_fcast2 = pd.DataFrame({'Year': [np.nan], 'Prophet ForecastValue Future': [np.nan], 'Prophet ForecastValue Validation': [np.nan], 'State': [s], 'ML Method': ['Prophet']})
        
################################################ Aggregating and Joining #######################################################

#     df_fcast1 = df_fcast1.groupby(['Year', 'State', 'ML Method'], as_index = False).agg({'Arima ForecastValue Future': 'sum', 'Arima ForecastValue Validation': 'sum'})
#     df_fcast2 = df_fcast2.groupby(['Year', 'State', 'ML Method'], as_index = False).agg({'Auto Arima ForecastValue Future': 'sum', 'Auto Arima ForecastValue Validation': 'sum'})
    df_fcast3 = df_fcast3.groupby(['Year', 'State', 'ML Method'], as_index = False).agg({'Prophet ForecastValue Future': 'sum', 'Prophet ForecastValue Validation': 'sum'})
        
    all_forecasts = df_fcast3.copy()
#     all_forecasts = df_fcast3.merge(df_fcast2[['Year', 'Auto Arima ForecastValue Future', 'Auto Arima ForecastValue Validation']], how ='left', on = 'Year').copy()
#     all_forecasts = pd.DataFrame(all_forecasts.merge(df_fcast1[['Year', 'Arima ForecastValue Future', 'Arima ForecastValue Validation']], how ='left', on = 'Year'))
        
        #Save predictions in df. First time this will fail and just give all_forecasts, after that will concat for each state
    try: 
        final_forecasts = pd.concat([final_forecasts, all_forecasts], ignore_index = True)
    except:
        final_forecasts = all_forecasts
        

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-pa

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/ae-j/opt/anaconda3/lib/

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

...

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/ae-j/opt/anaconda3/lib/

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/ae-j/opt/anaconda3/lib/

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/st

/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
/Users/ae-j/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/ae-j/opt/anaconda3/lib/

In [69]:
final_forecasts.shape

(969, 5)

In [70]:
#final_forecasts.to_csv('./data/time_series/final_v1.csv')

## Merging final forecasts with original data

Merging the final forecasts with original data. However our original dataframe is in the wrong format, so we import a new one that has a "states" row with the historical data in the correct format.

In [72]:
#Reading in our original data in the correct format and final_forecasts (which we exported above)
df_data = pd.read_csv('./data/time_series/hor_states.csv')

final_forecasts = pd.read_csv('./data/time_series/final_v1.csv')

In [73]:
df_data.head()

,Unnamed: 0,Year,State,fi
0,1.0,1/1/2010,Alabama,20.379104
1,2.0,1/1/2011,Alabama,19.292537
2,3.0,1/1/2012,Alabama,19.497412
3,4.0,1/1/2013,Alabama,18.569592
4,5.0,1/1/2014,Alabama,18.800626


In [74]:
##Setting consistent columns
df_data['Prophet ForecastValue Future'] = np.nan
df_data['Prophet ForecastValue Validation'] = np.nan

# df_data['Year'] = pd.to_datetime(df_data['Year'])
#Dropping Ml Method as it is not necessary
del final_forecasts['ML Method']

#adding this column to keep consistency across both before union
final_forecasts['fi'] = np.nan

KeyError: 'ML Method'

In [81]:
#Converting both to datetime so types are the same for the concat
df_data['Year'] = pd.to_datetime(df_data['Year'])
final_forecasts['Year'] = pd.to_datetime(final_forecasts['Year'])

In [82]:
df_data.head(1)

,Unnamed: 0,Year,State,fi,Prophet ForecastValue Future,Prophet ForecastValue Validation
0,1.0,2010-01-01,Alabama,20.379104,NaN,NaN


In [84]:
final_forecasts.head(1)

,Unnamed: 0,Year,State,Prophet ForecastValue Future,Prophet ForecastValue Validation,fi
0,0,2010-01-01,Alabama,19.888816,19.968676,NaN


In [85]:
## Reordering for concat

final_forecasts = final_forecasts[['Unnamed: 0', 'Year', 'State', 'fi', 'Prophet ForecastValue Future', 'Prophet ForecastValue Validation']]

In [100]:
#Concatening and dropping and renaming columns 
output_df = pd.concat([df_data, final_forecasts], axis = 0)

del output_df['Unnamed: 0']

output_df.rename(columns = {'fi': 'Food Insecurity Rate'}, inplace = True)

In [101]:
output_df.head(1)

,Year,State,Food Insecurity Rate,Prophet ForecastValue Future,Prophet ForecastValue Validation
0,2010-01-01,Alabama,20.379104,NaN,NaN


In [102]:
##Setting zeros to NaN for PowerBI

output_df['Prophet ForecastValue Future'][output_df['Prophet ForecastValue Future'] < 1] = np.nan
output_df['Prophet ForecastValue Validation'][output_df['Prophet ForecastValue Validation'] < 1] = np.nan
output_df['Food Insecurity Rate'][output_df['Food Insecurity Rate'] < 1] = np.nan


<ipython-input-102-8ebaf6f186c4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['Prophet ForecastValue Future'][output_df['Prophet ForecastValue Future'] < 1] = np.nan
<ipython-input-102-8ebaf6f186c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['Prophet ForecastValue Validation'][output_df['Prophet ForecastValue Validation'] < 1] = np.nan
<ipython-input-102-8ebaf6f186c4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [4]:
## Exporting to CSV
output_df.to_csv('./data/time_series/output_df.csv')

NameError: name 'output_df' is not defined